In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
CSV_FILE_PATH = '../merged_gyroscope_data.csv'
data = pd.read_csv(CSV_FILE_PATH)

In [3]:
def find_ids_of_activities(df):
    ids_of_activities = []
    id_of_activity = 0
    previous_activity = ''
    
    for index, row in df.iterrows():
        if previous_activity != row['activity_type']:
            id_of_activity = id_of_activity + 1
            previous_activity = row['activity_type']
        ids_of_activities.append(id_of_activity)
        
    return ids_of_activities

data['activity_id'] = find_ids_of_activities(data);
data = data[data['activity_type'] != 'pause'];

In [4]:
data['activity_id'].unique()

array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34,
       36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68,
       70, 72, 74, 75, 77, 79, 81, 83, 84, 85], dtype=int64)

In [8]:
def process_fft_result(fft_result):
    magnitudes = np.abs(fft_result);
    magnitudes = np.sum(magnitudes, 1);
    magnitudes = magnitudes * 2 / magnitudes.shape[0];
    magnitudes = np.array_split(magnitudes,2)[0];
    magnitudes = magnitudes[1:];
    return magnitudes;

def calculate_number_of_repeats(df):
    input = df[['y_axis', 'z_axis', 'ay_axis']]
    fft_result = np.fft.fftn(input, s=None, axes=None, norm=None);
    magnitudes = process_fft_result(fft_result);
    max = magnitudes.max();
    threshold = max / 1.45;
    top_frequencies = np.argwhere(magnitudes > threshold);
    predicted = top_frequencies.min();
    return predicted

error = 0
for id in data['activity_id'].unique() :
    df = data[data['activity_id'] == id];
    predicted = calculate_number_of_repeats(df);
    actual = df['number_of_repeats'].unique()[0];
    error = error + np.abs(predicted - actual);
    print("predicted / actual - %s / %s" % (predicted,actual));
    
print("MSE = %s" % (error / data['activity_id'].unique().shape[0]));

predicted / actual - 13 / 14
predicted / actual - 11 / 14
predicted / actual - 14 / 14
predicted / actual - 13 / 14
predicted / actual - 15 / 14
predicted / actual - 15 / 14
predicted / actual - 13 / 14
predicted / actual - 6 / 14
predicted / actual - 13 / 14
predicted / actual - 9 / 10
predicted / actual - 9 / 10
predicted / actual - 8 / 10
predicted / actual - 10 / 10
predicted / actual - 20 / 10
predicted / actual - 10 / 10
predicted / actual - 9 / 10
predicted / actual - 9 / 10
predicted / actual - 9 / 10
predicted / actual - 18 / 20
predicted / actual - 14 / 20
predicted / actual - 17 / 20
predicted / actual - 6 / 20
predicted / actual - 10 / 20
predicted / actual - 19 / 20
predicted / actual - 19 / 20
predicted / actual - 19 / 20
predicted / actual - 21 / 20
predicted / actual - 10 / 11
predicted / actual - 10 / 11
predicted / actual - 10 / 11
predicted / actual - 9 / 11
predicted / actual - 11 / 11
predicted / actual - 11 / 11
predicted / actual - 10 / 11
predicted / actual - 10